<a href="https://colab.research.google.com/github/CasCard/RIG-Works/blob/main/numba_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cv2
from google.colab import drive
from google.colab.patches import cv2_imshow

In [2]:
import numba
from numba import vectorize,njit,jit

In [3]:
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [ ]:
video1 = '/content/gdrive/My Drive/auv_files/redrodtrim.mp4'

In [4]:
import cv2
from skimage.color import rgb2hsv,hsv2rgb
import numpy as np

@jit
def  HSVStretching(sceneRadiance):
    sceneRadiance = np.uint8(sceneRadiance)
    height = len(sceneRadiance)
    width = len(sceneRadiance[0])
    img_hsv = rgb2hsv(sceneRadiance)
    h, s, v = cv2.split(img_hsv)
    img_s_stretching = global_stretching(s, height, width)
    img_v_stretching = global_stretching(v, height, width)

    labArray = np.zeros((height, width, 3), 'float64')
    labArray[:, :, 0] = h
    labArray[:, :, 1] = img_s_stretching
    labArray[:, :, 2] = img_v_stretching
    img_rgb = hsv2rgb(labArray) * 255

    # img_rgb = np.clip(img_rgb, 0, 255)

    return img_rgb

In [5]:
import numpy as np

@jit
def sceneRadianceRGB(sceneRadiance):

    sceneRadiance = np.clip(sceneRadiance, 0, 255)
    sceneRadiance = np.uint8(sceneRadiance)

    return sceneRadiance



In [6]:
import numpy as np

@jit
def global_stretching(img_L,height, width):
    I_min = np.min(img_L)
    I_max = np.max(img_L)
    I_mean = np.mean(img_L)


    # print('I_min',I_min)
    # print('I_max',I_max)
    # print('I_max',I_mean)

    array_Global_histogram_stretching_L = np.zeros((height, width))
    for i in range(0, height):
        for j in range(0, width):
            p_out = (img_L[i][j] - I_min) * ((1) / (I_max - I_min))
            array_Global_histogram_stretching_L[i][j] = p_out

    return array_Global_histogram_stretching_L



In [7]:
import numpy as np
import concurrent.futures
import copy

@jit
def histogram_general(r_array,height, width,input):
    length = height * width
    R_rray = []
    for i in range(height):
        for j in range(width):
            R_rray.append(r_array[i][j])
    R_rray.sort()
    I_min = int(R_rray[int(length / 500)])
    I_max = int(R_rray[-int(length / 500)])
    array_Global_histogram_stretching = np.zeros((height, width))
    if input == 0:
      for i in range(0, height):
        for j in range(0, width):
            if r_array[i][j] < I_min:
                # p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = I_min
            elif (r_array[i][j] > I_max):
                p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = 255
            else:
                p_out = int((r_array[i][j] - I_min) * ((255 - I_min) / (I_max - I_min)))+ I_min
                array_Global_histogram_stretching[i][j] = p_out

    elif input == 1:
      for i in range(0, height):
        for j in range(0, width):
            if r_array[i][j] < I_min:
                  p_out = r_array[i][j]
                  array_Global_histogram_stretching[i][j] = 0
            elif (r_array[i][j] > I_max):
                  p_out = r_array[i][j]
                  array_Global_histogram_stretching[i][j] = 255
            else:
                  p_out = int((r_array[i][j] - I_min) * ((255) / (I_max - I_min)))
                  array_Global_histogram_stretching[i][j] = p_out
    elif input == 2 :
      for i in range(0, height):
        for j in range(0, width):
            if r_array[i][j] < I_min:
                # p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = 0
            elif (r_array[i][j] > I_max):
                # p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = I_max
            else:
                p_out = int((r_array[i][j] - I_min) * ((I_max) / (I_max - I_min)))
                array_Global_histogram_stretching[i][j] = p_out

    return (array_Global_histogram_stretching)

def stretching(img):
    height = len(img)
    width = len(img[0])
    with concurrent.futures.ProcessPoolExecutor() as executor:
      input=[img[:, :, 2],img[:, :, 1],img[:, :, 0]]
      inp=[0,1,2]
      h=[height,height,height]
      w=[width,width,width]
    
      results=executor.map(histogram_general,input,h,w,inp)
      i=0
      for result in results:
        input[i]=result
        i=i+1
      
    return img




In [8]:
import numpy as np

@jit
def cal_equalisation(img,ratio):
    Array = img * ratio
    Array = np.clip(Array, 0, 255)
    return Array

@jit
def RGB_equalisation(img):
    i =0
    img = np.float32(img)
    avg_RGB = []
    for i in range(3):
        avg = np.mean(img[:,:,i])
        avg_RGB.append(avg)
    # print('avg_RGB',avg_RGB)
    a_r = avg_RGB[0]/avg_RGB[2]
    a_g =  avg_RGB[0]/avg_RGB[1]
    ratio = [0,a_g,a_r]
    for i in range(1,3):
        img[:,:,i] = cal_equalisation(img[:,:,i],ratio[i])
    return img






In [10]:
import os
import numpy as np
import cv2
import natsort
import xlwt
import datetime


np.seterr(over='ignore')
if __name__ == '__main__':
    pass

folder = "/content/gdrive/My Drive"

# path = sett
pathinp = folder + "/auv_files/test_images"
pathout = folder + "/auv_files/out_test_images"
filesinp = os.listdir(pathinp)
filesoup = os.listdir(pathout)
length = len(filesinp)
for i in range(length):
    imginp = filesinp[i]
    imgdir = pathinp + "/" + imginp
    # print(filesinp[i])
    print('********    file   ********',imgdir)
    # img = cv2.imread('InputImages/' + file)
    import timeit
    start = timeit.default_timer()
    img = cv2.imread(imgdir)
    print(len(img))
    #img = cv2.resize(img,(256,256))
    #cv2_imshow(img)
    sceneRadiance = RGB_equalisation(img)
    sceneRadiance = stretching(sceneRadiance)
    # # cv2.imwrite(folder + '/OutputImages/' + Number + 'Stretched.jpg', sceneRadiance)
    sceneRadiance = HSVStretching(sceneRadiance)
    sceneRadiance = sceneRadianceRGB(sceneRadiance)
    cv2.imwrite('/content/gdrive/My Drive/auv_files/out_test_images/'+ "oup" + imginp,sceneRadiance)
    #cv2_imshow(sceneRadiance)
    stop = timeit.default_timer()
    print('Time: ', stop - start)  



rod4.jpg
********    file   ******** /content/gdrive/My Drive/auv_files/test_images/rod4.jpg
1080
1080
Time:  62.276370844999974
rod3.jpg
********    file   ******** /content/gdrive/My Drive/auv_files/test_images/rod3.jpg
1080
1080
Time:  63.34634116400002
rod2.jpg
********    file   ******** /content/gdrive/My Drive/auv_files/test_images/rod2.jpg
1080
1080


KeyboardInterrupt: ignored

In [13]:
import os
import numpy as np
import cv2
import natsort
import xlwt
import datetime
import timeit

folder = "/content/gdrive/My Drive"
pathinp = folder + "/auv_files/test_images"
pathout = folder + "/auv_files/out_test_images"
filesinp = os.listdir(pathinp)
filesoup = os.listdir(pathout)
length = len(filesinp)
print(filesinp)

['rod4.jpg', 'rod3.jpg', 'rod2.jpg', 'rod1.jpg']


In [14]:
starti = timeit.default_timer()
def process_image(imginp):
  imgdir = pathinp + "/" + imginp
  img = cv2.imread(imgdir)

  start = timeit.default_timer()
  sceneRadiance = RGB_equalisation(img)
  stop = timeit.default_timer()
  print('RGB_equalisation: ', stop - start)

  start = timeit.default_timer()
  sceneRadiance = stretching(sceneRadiance)
  stop = timeit.default_timer()
  print('stretching: ', stop - start)

  start = timeit.default_timer()
  sceneRadiance = HSVStretching(sceneRadiance)
  stop = timeit.default_timer()
  print('HSVStretching: ', stop - start)

  start = timeit.default_timer()
  sceneRadiance = sceneRadianceRGB(sceneRadiance)
  stop = timeit.default_timer()
  print('sceneRadianceRGB: ', stop - start)
  return sceneRadiance

i=0
with concurrent.futures.ProcessPoolExecutor() as executor:
  results=executor.map(process_image,filesinp)
  for result in results:
    cv2.imwrite('/content/gdrive/My Drive/auv_files/out_test_images/'+ str(i) +'.jpg',result)
    i=i+1

stopi = timeit.default_timer()
print('Time: ', stopi - starti) 


<ipython-input-8-b6a9f4654ecb>:9: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "RGB_equalisation" failed type inference due to: Invalid use of Function(<built-in function getitem>) with argument(s) of type(s): (float32, Tuple(slice<a:b>, slice<a:b>, int64))
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
In definition 2:
    All templates rejected with literals.
In definition 3:
    All templates rejected without literals.
In definition 4:
    All templates rejected with literals.
In definition 5:
    All templates rejected without literals.
In definition 6:
    All templates rejected with literals.
In definition 7:
    All templates rejected without literals.
In definition 8:
    All templates rejected with literals.
In definition 9:
    All templates rejected without literals.
In definition 10:
    All templates rejected with literals.
In de

RGB_equalisation:  1.9022086109999918
RGB_equalisation:  1.9060434740000574
stretching:  8.528960590999986


<ipython-input-4-034a3805a2b2>:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "HSVStretching" failed type inference due to: Untyped global name 'rgb2hsv': cannot determine Numba type of <class 'function'>

File "<ipython-input-4-034a3805a2b2>", line 10:
def  HSVStretching(sceneRadiance):
    <source elided>
    width = len(sceneRadiance[0])
    img_hsv = rgb2hsv(sceneRadiance)
    ^

  @jit


stretching:  8.583574650999935


<ipython-input-4-034a3805a2b2>:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "HSVStretching" failed type inference due to: Untyped global name 'rgb2hsv': cannot determine Numba type of <class 'function'>

File "<ipython-input-4-034a3805a2b2>", line 10:
def  HSVStretching(sceneRadiance):
    <source elided>
    width = len(sceneRadiance[0])
    img_hsv = rgb2hsv(sceneRadiance)
    ^

  @jit
/usr/local/lib/python3.6/dist-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "HSVStretching" was compiled in object mode without forceobj=True.

File "<ipython-input-4-034a3805a2b2>", line 6:
@jit
def  HSVStretching(sceneRadiance):
^

  state.func_ir.loc))
/usr/local/lib/python3.6/dist-packages/numba/object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata

HSVStretching:  3.5898920299999872


<ipython-input-5-e3a0d4f4f5f5>:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "sceneRadianceRGB" failed type inference due to: Use of unsupported NumPy function 'numpy.clip' or unsupported use of the function.

File "<ipython-input-5-e3a0d4f4f5f5>", line 6:
def sceneRadianceRGB(sceneRadiance):
    <source elided>

    sceneRadiance = np.clip(sceneRadiance, 0, 255)
    ^

[1] During: typing of get attribute at <ipython-input-5-e3a0d4f4f5f5> (6)

File "<ipython-input-5-e3a0d4f4f5f5>", line 6:
def sceneRadianceRGB(sceneRadiance):
    <source elided>

    sceneRadiance = np.clip(sceneRadiance, 0, 255)
    ^

  @jit


HSVStretching:  3.5466308699999445


<ipython-input-5-e3a0d4f4f5f5>:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "sceneRadianceRGB" failed type inference due to: Use of unsupported NumPy function 'numpy.clip' or unsupported use of the function.

File "<ipython-input-5-e3a0d4f4f5f5>", line 6:
def sceneRadianceRGB(sceneRadiance):
    <source elided>

    sceneRadiance = np.clip(sceneRadiance, 0, 255)
    ^

[1] During: typing of get attribute at <ipython-input-5-e3a0d4f4f5f5> (6)

File "<ipython-input-5-e3a0d4f4f5f5>", line 6:
def sceneRadianceRGB(sceneRadiance):
    <source elided>

    sceneRadiance = np.clip(sceneRadiance, 0, 255)
    ^

  @jit
/usr/local/lib/python3.6/dist-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "sceneRadianceRGB" was compiled in object mode without forceobj=True.

File "<ipython-input-5-e3a0d4f4f5f5>", line 4:
@jit
def sceneRadianceRGB(sceneRadiance):
^

  state.func_ir.loc))
/usr/local/lib/python3.6/dist-packages/numba/

sceneRadianceRGB:  0.13368530900004316
sceneRadianceRGB:  0.12802086900001086
RGB_equalisation:  0.032632005999971625
RGB_equalisation:  0.0770194189999529
stretching:  8.485639644000003
stretching:  8.580746754000074
HSVStretching:  1.610987041000044
sceneRadianceRGB:  0.06652180000003227
HSVStretching:  1.5150413750000098
sceneRadianceRGB:  0.024156963999985237
Time:  25.61443561100009


In [ ]:
# cap = cv2.VideoCapture(video1)
# ret,frame=cap.read()
# count=0
# out = cv2.VideoWriter('/content/gdrive/My Drive/auv_files/output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (frame.shape[1], frame.shape[0]))
# while cap.isOpened():
#     import timeit
#     starti = timeit.default_timer()
#     ret, frame = cap.read()

#     # check for successfulness of cap.read()
#     if not ret: break
#     start = timeit.default_timer()
#     sceneRadiance = RGB_equalisation(frame)
#     stop = timeit.default_timer()
#     print('RGB_equalisation: ', stop - start ,count)

#     start = timeit.default_timer()
#     sceneRadiance = stretching(sceneRadiance)
#     stop = timeit.default_timer()
#     print('stretching ', stop - start ,count)
#     # # cv2.imwrite(folder + '/OutputImages/' + Number + 'Stretched.jpg', sceneRadiance)
#     start = timeit.default_timer()
#     sceneRadiance = HSVStretching(sceneRadiance)
#     stop = timeit.default_timer()
#     print('HSVStretching: ', stop - start ,count)

#     start = timeit.default_timer()
#     sceneRadiance = sceneRadianceRGB(sceneRadiance)
#     stop = timeit.default_timer()
#     print('sceneRadianceRGB: ', stop - start ,count)
#     # Save the video
#     out.write(sceneRadiance)
#     stopi = timeit.default_timer()
#     print('Time per image ', stopi - starti ,count)
    # cv2_imshow(image) # Note cv2_imshow, not cv2.imshow

    # cv2.waitKey()

# cv2.destroyAllWindows()
# cap.release()
# out.release()

In [ ]:
import os
import numpy as np
import cv2
import natsort
import xlwt
import datetime

np.seterr(over='ignore')
if __name__ == '__main__':
    pass


cap = cv2.VideoCapture(video1)
ret,frame=cap.read()
count=0
frames=[]
out = cv2.VideoWriter('/content/gdrive/My Drive/auv_files/output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1], frame.shape[0]))
while cap.isOpened():
    # import timeit
    # starti = timeit.default_timer()
    ret, frame = cap.read()
    if ret:
      frames.append(frame)
    if not ret: break

cap.release()
print(len(frames))
# out.release()

222


In [ ]:
# check for successfulness of cap.read()
starti = timeit.default_timer()
def process_image(img_frame):
    start = timeit.default_timer()
    sceneRadiance = RGB_equalisation(img_frame)
    stop = timeit.default_timer()
    print('RGB_equalisation: ', stop - start ,count)

    start = timeit.default_timer()
    sceneRadiance = stretching(sceneRadiance)
    stop = timeit.default_timer()
    print('stretching ', stop - start ,count)
    # # cv2.imwrite(folder + '/OutputImages/' + Number + 'Stretched.jpg', sceneRadiance)
    start = timeit.default_timer()
    sceneRadiance = HSVStretching(sceneRadiance)
    stop = timeit.default_timer()
    print('HSVStretching: ', stop - start ,count)

    start = timeit.default_timer()
    sceneRadiance = sceneRadianceRGB(sceneRadiance)
    stop = timeit.default_timer()
    print('sceneRadianceRGB: ', stop - start ,count)
    # Save the video
    # out.write(sceneRadiance)
    
    return sceneRadiance 


with concurrent.futures.ProcessPoolExecutor() as executor:
  results=executor.map(process_image,frames)
  for result in results:
    out.write(result)
out.release()
stopi = timeit.default_timer()
print('Time per image ', stopi - starti)

RGB_equalisation:  0.010079515999677824 0
RGB_equalisation:  0.017228011000042898 0
stretching  15.508185057000446 0
stretching  15.593584006000128 0
HSVStretching:  1.644942419000472 0
sceneRadianceRGB:  0.008886718999747245 0
RGB_equalisation:  0.005787452999356901 0
HSVStretching:  1.6390013850004834 0
sceneRadianceRGB:  0.006903916000737809 0
RGB_equalisation:  0.007542534000094747 0
stretching  14.967041370000516 0
stretching  14.973263818000305 0
HSVStretching:  1.5376676619998761 0
sceneRadianceRGB:  0.0063085649999266025 0
RGB_equalisation:  0.0047076359996935935 0
HSVStretching:  1.5365077209999072 0
sceneRadianceRGB:  0.005856105999555439 0
RGB_equalisation:  0.008648761000586092 0
stretching  14.556209244999991 0
stretching  14.644221217999984 0
HSVStretching:  1.4975958649993117 0
sceneRadianceRGB:  0.005581950999840046 0
RGB_equalisation:  0.00818588499987527 0
HSVStretching:  1.5089331070003027 0
sceneRadianceRGB:  0.005579683000178193 0
RGB_equalisation:  0.0054682320005

In [ ]:
import numba